In [1]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import
from __future__ import unicode_literals

import os
import sys

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl

%matplotlib notebook

In [2]:
try:
    import seaborn as sns
    sns.set()
    sns.set_style("whitegrid")
    sns.set_context("poster")
except ImportError as e:
    print("Cannot import seaborn.")
    print("Consider installing it for nice plot !")

mpl.rcParams['figure.figsize'] = [12.0, 9.0]
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100

mpl.rcParams['font.size'] = 22
mpl.rcParams['axes.labelsize'] = 18
mpl.rcParams['ytick.labelsize'] = 18
mpl.rcParams['xtick.labelsize'] = 18
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'


# Introduction

# DBscan

http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html

In [86]:
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [96]:
from tracker import BaseTracker

class DBSCANTracker(BaseTracker):
    def __init__(self, eps=0.5, min_samples=3, metric='euclidean', 
                 algorithm='auto', leaf_size=30, p=None, n_jobs=1, verbose=1):
        super().__init__()
        self.dbscan = DBSCAN(eps=eps, min_samples=min_samples,
                             metric=metric,algorithm=algorithm, 
                             leaf_size=leaf_size, p=p, n_jobs=n_jobs)
        self.eps = eps
        self.min_samples = min_samples
        self.metric = metric
        self.algorithm = algorithm
        self.leaf_size = leaf_size
        self.p = p
        self.n_jobs = n_jobs
        self.verbose = verbose
        
    def fit_generator(self, event_generator):
        pass

    def predict_generator(self, event_generator):
        sub_list = [self.predict_one_event(event_id, event) for event_id, event in event_generator]
        submission = pd.concat(sub_list)
        return submission
    
    def predict_one_event(self, event_id, event):
        if self.verbose:
            print(event_id)
        event['r'] = np.hypot(event['x'], event['y'])
        event['phi'] = np.arctan2(event['y'], event['x'])
        X = event[['r', 'phi', 'z']]
#         X = event[['phi', 'z']]
#         X = event['phi'].values.reshape(-1, 1)
#         X = StandardScaler().fit_transform(X)
        labels = self.dbscan.fit_predict(X)

        sub = pd.DataFrame(data=np.column_stack((event.hit_id.values, labels)),
                                  columns=["hit_id", "track_id"]).astype(int)
        sub['event_id'] = event_id
        return sub

        

# Tracking on the training set 

In [97]:
from dataset import load_hit_generator

data_path = "./public_data/training/"
event_generator = load_hit_generator(data_path)

In [98]:
tracker = DBSCANTracker(n_jobs=5)

In [99]:
submission = tracker.predict_generator(event_generator)

50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199


# Scoring

In [100]:
solution = pd.read_csv("./public_data/training/solution.csv")

In [101]:
from score import compute_score

In [102]:
%%time
score_mean, score_std, n_purity_reco, n_purity_truth = compute_score(submission, solution)

CPU times: user 19.3 s, sys: 56 ms, total: 19.4 s
Wall time: 19.3 s


In [103]:
print('score mean:   {:f}'.format(score_mean))
print('score stddev: {:f}'.format(score_std))
print('purity_reco: {:d}'.format(n_purity_reco))
print('purity_truth: {:d}'.format(n_purity_truth))


score mean:   0.000000
score stddev: 0.000000
purity_reco: 105
purity_truth: 41


In [104]:
print(len(submission['track_id'].unique()), 'predicted particles')
print(len(solution['particle_id'].unique()), 'true particles')


474 predicted particles
21866 true particles


# Write Submission

In [ ]:
data_path = "./public_data/validation/"
event_generator = load_hit_generator(data_path)
submission = tracker.predict_generator(event_generator)

In [ ]:
submission.to_csv('./submission.csv')

Now just zip the file (**ONLY THE csv FILE ! NOT THE DIRECTORY**) and you can submit it to codalab.

The submission file has to be name "submission.csv"